In [2]:
from __future__ import division
from __future__ import print_function

import argparse
import os, sys, time
import shutil
import random
import torch
import numpy as np
import gc
import torchvision.models

import data_util
import scene_dataloader
import model as model_util
import loss as loss_util
import style

from utils.raycast_rgbd.raycast_rgbd import RaycastRGBD
from utils.raycast_rgbd.raycast_rgbd import RaycastOcc
from utils.depth_utils.depth_utils import Depth2Normals
import utils.color_utils_cpu.color_utils as color_utils

COLOR_MEAN = [0.485, 0.456, 0.406] # for vgg19
COLOR_STD = [0.229, 0.224, 0.225]

GAN_TYPES = ['hinge', 'vanilla', 'wgan', 'wgan_gp']
COLOR_SPACES = ['rgb', 'lab']


%load_ext autoreload
%autoreload 2

In [ ]:
# specify gpu
os.environ['CUDA_VISIBLE_DEVICES']=str(args.gpu)
torch.backends.cudnn.benchmark = True

# style part
model_style = None
raycaster_rgbd = None
gan_loss = None

In [ ]:
truncation = 3

model_style = torchvision.models.vgg19(pretrained=True).cuda().eval()
model_style = style.Model(model_style, COLOR_MEAN, COLOR_STD)
ray_increment = 0.3 * args.truncation
thresh_sample_dist = 50.5 * ray_increment
max_num_locs_per_sample = 640000
raycast_depth_max = 6.0
raycaster_rgbd = RaycastRGBD(args.batch_size, args.input_dim, args.style_width, args.style_height, depth_min=0.1/args.voxelsize, depth_max=raycast_depth_max/args.voxelsize, thresh_sample_dist=thresh_sample_dist, ray_increment=ray_increment, max_num_locs_per_sample=max_num_locs_per_sample)
depth2normals = Depth2Normals(args.batch_size, args.style_width, args.style_height, depth_min=0.1/args.voxelsize, depth_max=raycast_depth_max/args.voxelsize, max_num_fill_iters=args.max_depth_fill_iters)
    